### Business Problem : 
 An established resto-bar owner wants to expand his business by setting up another branch in Toronto. He approaches a data analytics firm where as a data scientist we will help him gain some insights on location preferences and scope out the competition. He would also like to know the existing places in and around Toronto where he can access foot traffic, parking and accessibility.



### Data : 
 For the above business problem we will collect the data of all the bars and restaurants in Toronto. The data would contain longitude and latitude information of each location and this can be obtained from foursquare API. This collected data will be used for clustering analysis  to find the density of bars and help the client in finding the right place to set up his new branch of resto-bar.

In [1]:

!pip install bs4

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.


### Importing Packages

In [2]:

from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import numpy as np

### Extracting data from wikipedia

In [3]:
tr_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data= rq.get(tr_url)
html_txt=data.text

### Webscraping the text to extract table contents from the html usig BeautifulSoup 

In [4]:
soup = BeautifulSoup(html_txt,"html5lib")

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df.shape

(103, 3)

### Load Geospatial coordinates data into a data frame

In [7]:
geo_cord=pd.read_csv('Geospatial_Coordinates.csv')
geo_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the data with existing postal code data 

In [8]:
geo_data=df.merge(geo_cord, on='Postal Code')
geo_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [9]:
geo_data.shape

(103, 5)

In [10]:
conda install -c conda-forge folium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [11]:
import folium 

In [12]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [13]:
from geopy.geocoders import Nominatim

### Get Latitude and Longitude of Toronto

In [14]:

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Toronto')
lat = location.latitude
lon = location.longitude
print('The coordinate of Toronto are {}, {}.'.format(lat, lon))

The coordinate of Toronto are 43.6534817, -79.3839347.


### Create Map and add markers

In [15]:

to_map = folium.Map(location=[lat, lon], zoom_start=10)

for lat, lng, borough, neighborhood in zip(geo_data['Latitude'], geo_data['Longitude'], geo_data['Borough'], geo_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#ff0000',
        fill_opacity=0.7,
        parse_html=False).add_to(to_map)  
    
to_map

# Extract Data from Foursquare API

In [16]:
#code = X5EAGC5PUIL53ZHGJYFBXQFSUZL4JW2UPQZOQYGMAT11FL5X#_=_
#"access_token":"0ZJC4LGDSFOLTCZJ3GSYF0YOXBIWSNSO4OKVQIXZEAT5XCKQ"

CLIENT_ID = 'OMEBNUL4PAWEID510BUAJ2AEPECSDASR2HVXW4HMDBEXUC05' # your Foursquare ID
CLIENT_SECRET = '5VZPE21JJZRL3YBKMWB5ZD1FIZNIOIU12KY1HHMZNI1NFE4Y' # your Foursquare Secret
ACCESS_TOKEN = '0ZJC4LGDSFOLTCZJ3GSYF0YOXBIWSNSO4OKVQIXZEAT5XCKQ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OMEBNUL4PAWEID510BUAJ2AEPECSDASR2HVXW4HMDBEXUC05
CLIENT_SECRET:5VZPE21JJZRL3YBKMWB5ZD1FIZNIOIU12KY1HHMZNI1NFE4Y


In [17]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = rq.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
tor_near_values = getNearbyVenues(names=geo_data['Neighborhood'],
                                   latitudes=geo_data['Latitude'],
                                   longitudes=geo_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [19]:
tor_near_values.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",2,2,2,2,2,2
Woburn,4,4,4,4,4,4
Woodbine Heights,8,8,8,8,8,8


# Analyzing the neighborhood

In [20]:

# one hot encoding
tor_onehot = pd.get_dummies(tor_near_values[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_near_values['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_grp = tor_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grp.shape)
toronto_grp

(101, 237)


,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0
99,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:

top_ven = 5

for hood in toronto_grp['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grp[toronto_grp['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_ven))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1  Sandwich Place  0.14
2             Gym  0.14
3             Pub  0.14
4     Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                Pizza Place  0.05
3                      Diner  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1        Sandwich Place

4  Medical Center   0.0


----Humewood-Cedarvale----
          venue  freq
0         Field  0.25
1  Hockey Arena  0.25
2          Park  0.25
3         Trail  0.25
4         Motel  0.00


----India Bazaar, The Beaches West----
            venue  freq
0     Pizza Place  0.10
1            Park  0.10
2  Sandwich Place  0.10
3    Liquor Store  0.05
4       Pet Store  0.05


----Kennedy Park, Ionview, East Birchmount Park----
               venue  freq
0  Convenience Store  0.25
1         Hobby Shop  0.25
2        Coffee Shop  0.25
3   Department Store  0.25
4      Metro Station  0.00


----Kensington Market, Chinatown, Grange Park----
                   venue  freq
0                   Café  0.10
1  Vietnamese Restaurant  0.07
2   Caribbean Restaurant  0.07
3                 Bakery  0.07
4            Fish Market  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0               Park  0.33
1  Mobile Phone Shop  0.33
2     Sandwic

                 venue  freq
0                 Café  0.17
1       Sandwich Place  0.07
2                  Bar  0.07
3  Japanese Restaurant  0.07
4            Bookstore  0.07


----Victoria Village----
                   venue  freq
0  Portuguese Restaurant   0.2
1      French Restaurant   0.2
2           Intersection   0.2
3            Coffee Shop   0.2
4           Hockey Arena   0.2


----West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale----
                       venue  freq
0  Middle Eastern Restaurant   0.5
1                     Bakery   0.5
2                Yoga Studio   0.0
3                     Market   0.0
4             Massage Studio   0.0


----Westmount----
                venue  freq
0      Sandwich Place  0.12
1  Chinese Restaurant  0.12
2          Playground  0.12
3         Pizza Place  0.12
4      Discount Store  0.12


----Weston----
                      venue  freq
0         Convenience Store   1.0
1               Yoga Studio   0.0
2              

In [59]:
def return_most_common_venues(row, top_ven):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_ven]
top_ven = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_ven):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neigh_ven = pd.DataFrame(columns=columns)
neigh_ven['Neighborhood'] = toronto_grp['Neighborhood']

for ind in np.arange(toronto_grp.shape[0]):
    neigh_ven.iloc[ind, 1:] = return_most_common_venues(toronto_grp.iloc[ind, :], top_ven)

neigh_ven.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Clothing Store,Latin American Restaurant,Skating Rink,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Pub,Skating Rink,Sandwich Place,Coffee Shop,Dog Run,Donut Shop,Distribution Center,Dance Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Deli / Bodega,Sushi Restaurant,Supermarket,Bridal Shop,Diner,Shopping Mall,Sandwich Place
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Pharmacy,Pub,Juice Bar,Butcher,Fast Food Restaurant,Café,Indian Restaurant


# Clustering Analysis

In [47]:
from sklearn.cluster import KMeans


In [60]:
# set number of clusters
kclusters = 5

toronto_grp_clustering = toronto_grp.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grp_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 4], dtype=int32)

In [61]:
# add clustering labels
neigh_ven.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))
neigh_ven['Cluster Labels'] = pd.to_numeric(neigh_ven['Cluster Labels'])
neigh_ven = neigh_ven.astype({"Cluster Labels": int})
toronto_geo = geo_data

# merge toronto_grp with geo_data to add latitude/longitude for each neighborhood
toronto_geo = toronto_geo.join(neigh_ven.set_index('Neighborhood'), on='Neighborhood')

toronto_geo.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Bus Stop,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Portuguese Restaurant,Coffee Shop,French Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Gym / Fitness Center,Historic Site,Restaurant,Pub,Performing Arts Venue,Mexican Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Boutique,Vietnamese Restaurant,Coffee Shop,Distribution Center,Dim Sum Restaurant,Diner
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Gym,Hobby Shop,Bar,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place


In [63]:
toronto_geo["Cluster Labels"].fillna( method ='ffill', inplace = True)
toronto_geo["Cluster Labels"] = toronto_geo["Cluster Labels"].astype(int)
toronto_geo.dtypes

Postal Code                object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [65]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_geo['Latitude'], toronto_geo['Longitude'], toronto_geo['Neighborhood'], toronto_geo['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1 data 

In [66]:
toronto_geo.loc[toronto_geo['Cluster Labels'] == 0, toronto_geo.columns[[1] + list(range(5, toronto_geo.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,0,Park,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
52,North York,0,Park,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


# Cluster 2 data 

In [67]:
toronto_geo.loc[toronto_geo['Cluster Labels'] == 1, toronto_geo.columns[[1] + list(range(5, toronto_geo.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Intersection,Portuguese Restaurant,Coffee Shop,French Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,Downtown Toronto,1,Coffee Shop,Bakery,Park,Breakfast Spot,Gym / Fitness Center,Historic Site,Restaurant,Pub,Performing Arts Venue,Mexican Restaurant
3,North York,1,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Boutique,Vietnamese Restaurant,Coffee Shop,Distribution Center,Dim Sum Restaurant,Diner
4,Queen's Park,1,Coffee Shop,Sushi Restaurant,Gym,Hobby Shop,Bar,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place
5,Etobicoke,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,1,Café,Coffee Shop,Restaurant,Seafood Restaurant,Hotel,Gym / Fitness Center,Art Gallery,Speakeasy,Japanese Restaurant,Steakhouse
98,Etobicoke,1,Pool,River,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
99,Downtown Toronto,1,Bookstore,Hobby Shop,Dance Studio,Sushi Restaurant,Ice Cream Shop,Indian Restaurant,Martial Arts School,Restaurant,Italian Restaurant,Beer Bar
100,East Toronto Business,1,Gym / Fitness Center,Spa,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden,Garden Center


# Cluster 3 data 

In [68]:
toronto_geo.loc[toronto_geo['Cluster Labels'] == 2, toronto_geo.columns[[1] + list(range(5, toronto_geo.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,2,Baseball Field,Food Truck,Women's Store,Falafel Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
57,North York,2,Baseball Field,Women's Store,Farmers Market,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
101,Etobicoke,2,Baseball Field,Women's Store,Farmers Market,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


# Cluster 4 data 

In [69]:
toronto_geo.loc[toronto_geo['Cluster Labels'] == 3, toronto_geo.columns[[1] + list(range(5, toronto_geo.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,York,3,Convenience Store,Women's Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


# Cluster 5 data 

In [70]:
toronto_geo.loc[toronto_geo['Cluster Labels'] == 4, toronto_geo.columns[[1] + list(range(5, toronto_geo.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Bus Stop,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
10,North York,4,Playground,Park,Bakery,Japanese Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
16,York,4,Field,Park,Trail,Hockey Arena,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
21,York,4,Park,Women's Store,Pool,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
35,East York/East Toronto,4,Coffee Shop,Park,Convenience Store,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
40,North York,4,Airport,Park,Women's Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
49,North York,4,Basketball Court,Bakery,Park,Trail,Construction & Landscaping,Dog Run,Diner,Discount Store,Distribution Center,Women's Store
61,Central Toronto,4,Park,Swim School,Dim Sum Restaurant,Bus Line,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
66,North York,4,Park,Convenience Store,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
77,Etobicoke,4,Park,Mobile Phone Shop,Sandwich Place,Women's Store,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
